# Preços de Medicamentos - Consumidor
### Nesse projeto irei criar uma API para saber de fato de quanto foi o aumento no preço dos médicamento nesse último reajuste no brasil.

<img src="https://i.imgur.com/W7DMmnl.jpeg" />

Os dados representam a lista de preços de Medicamentos , contemplando o preço Fábrica, ou preço **fabricante (PF)**, que é o preço máximo que pode ser praticado pelas empresas produtoras ou importadoras do produto e pelas empresas distribuidoras. O PF indica o preço máximo permitido para venda a farmácias e drogarias e o **Preço Máximo ao Consumidor (PMC)** indica o preço teto de venda ao consumidor.

Fonte dos dados: https://dados.anvisa.gov.br/dados

### Agência Nacional de Vigilância Sanitária - ANVISA

Criada pela Lei nº 9.782, de 26 de janeiro 1999, a Agência Nacional de Vigilância Sanitária (Anvisa) é uma autarquia sob regime especial, que tem sede e foro no Distrito Federal, e está presente em todo o território nacional por meio das coordenações de portos, aeroportos, fronteiras e recintos alfandegados.

Tem por finalidade institucional promover a proteção da saúde da população, por intermédio do controle sanitário da produção e consumo de produtos e serviços submetidos à vigilância sanitária, inclusive dos ambientes, dos processos, dos insumos e das tecnologias a eles relacionados, bem como o controle de portos, aeroportos, fronteiras e recintos alfandegados.

- Data da criação do banco de dados: 31 de maio de 2017
- Penúltima atualização: 13 de outubro de 2020 para o DataFrame anterior
- Ultima atualização: 08 de fevereiro de 2021 para o DataFrame reajuste

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import folium
%matplotlib inline



import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 999)

In [2]:
import mysql.connector
import pandas as a

conn = mysql.connector.connect(host="localhost", user="root", passwd="1234", db="preco_medicamentos_brasil")
anterior = a.read_sql_query("SELECT * FROM TA_PRECO_MEDICAMENTO_10_2020" 
                      ,conn)


reajuste = a.read_sql_query("SELECT * FROM TA_PRECO_MEDICAMENTO_03_2021" 
                      ,conn)

In [3]:
anterior.head()

,SUBSTANCIA,CNPJ,LABORATORIO,CODIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTACAO,CLASSE TERAPEUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PRECO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRICAO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANALISE RECURSAL,LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS),COMERCIALIZACAO 2019,TARJA
0,ACETATO DE DEXAMETASONA;CLOTRIMAZOL,18.459.628/0001-15,BAYER S.A.,"5,39E+14","1,71E+12","7,89E+12",-,-,BAYCUTEN N,"10 MG/G + 0,443 MG/G CREM DERM CT BG AL X 40 G",D7B2 - CORTICOESTERÓIDES ASSOCIADOS A ANTIMICO...,Novo,Regulado,"19,74","22,11","25,55","27,32","23,78","27,51","23,93","27,71","27,71","28,51","29,66","34,13","36,42","32,87","36,66","33,08","36,92","33,28","37,95",Năo,Năo,Năo,Năo,,Negativa,Sim,Tarja Vermelha
1,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACĘUTICA LTDA,"5,05E+14","1,02E+12","7,90E+12",-,-,ORENCIA,125 MG/ML SOL INJ CT 1 SER PREENCHIDA,M1C - AGENTES ANTI-REUMÁTICOS ESPECÍFICOS,Biológico,Regulado,"1228,49","1228,49","1396,01","1480,11","1480,11","1489,08","1489,08","1498,16","1498,16","1535,61","1698,32","1929,9","2046,17","2046,17","2058,57","2058,57","2071,12","2071,12","2122,89",Năo,Năo,Năo,Năo,,Positiva,Năo,Tarja Vermelha
2,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACĘUTICA LTDA,"5,05E+14","1,02E+12","7,90E+12",-,-,ORENCIA,125 MG/ML SOL INJ CT 4 SER PREENC VD TRANS + D...,M1C - AGENTES ANTI-REUMÁTICOS ESPECÍFICOS,Biológico,Regulado,"4914,04","4914,04","5584,14","5920,53","5920,53","5956,41","5956,41","5992,73","5992,73","6142,55","6793,37","7719,75","8184,79","8184,79","8234,39","8234,39","8284,6","8284,6","8491,72",Năo,Năo,Sim,Năo,,Positiva,Sim,Tarja Vermelha (*)
3,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACĘUTICA LTDA,"5,05E+14","1,02E+12","7,90E+12",-,-,ORENCIA,250 MG PO LIOF INJ 1 CT 1 FA + SER DESCARTÁVEL,M1C - AGENTES ANTI-REUMÁTICOS ESPECÍFICOS,Biológico,Regulado,"1547,45","1547,45","1758,47","1864,4","1864,4","1875,7","1875,7","1887,13","1887,13","1934,31",,,,,,,,,,Sim,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha
4,ABCIXIMABE,51.780.468/0001-87,JANSSEN-CILAG FARMACĘUTICA LTDA,"5,15E+14","1,12E+12","7,90E+12",-,-,REOPRO,2 MG/ML SOL INJ CT FA VD INC X 5 ML,"B1C3 - INIBIDORES DA AGREGAÇĂO PLAQUETÁRIA, AN...",-,Regulado,"2003,06","2003,06","2276,21","2413,32","2413,32","2427,95","2427,95","2442,76","2442,76","2503,83",,,,,,,,,,Sim,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha (*)


In [4]:
reajuste.head()

,SUBSTANCIA (REAJUSTE),CNPJ (REAJUSTE),LABORATORIO (REAJUSTE),CODIGO GGREM (REAJUSTE),REGISTRO (REAJUSTE),EAN 1 (REAJUSTE),EAN 2 (REAJUSTE),EAN 3 (REAJUSTE),PRODUTO (REAJUSTE),APRESENTACAO (REAJUSTE),CLASSE TERAPEUTICA (REAJUSTE),TIPO DE PRODUTO (STATUS DO PRODUTO) (REAJUSTE),REGIME DE PRECO (REAJUSTE),PF Sem Impostos (REAJUSTE),PF 0% (REAJUSTE),PF 12% (REAJUSTE),PF 17% (REAJUSTE),PF 17% ALC (REAJUSTE),"PF 17,5% (REAJUSTE)","PF 17,5% ALC (REAJUSTE)",PF 18% (REAJUSTE),PF 18% ALC (REAJUSTE),PF 20% (REAJUSTE),PMC 0% (REAJUSTE),PMC 12% (REAJUSTE),PMC 17% (REAJUSTE),PMC 17% ALC (REAJUSTE),"PMC 17,5% (REAJUSTE)","PMC 17,5% ALC (REAJUSTE)",PMC 18% (REAJUSTE),PMC 18% ALC (REAJUSTE),PMC 20% (REAJUSTE),RESTRICAO HOSPITALAR (REAJUSTE),CAP (REAJUSTE),CONFAZ 87 (REAJUSTE),ICMS 0% (REAJUSTE),ANALISE RECURSAL (REAJUSTE),LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS) (REAJUSTE),COMERCIALIZACAO 2019 (REAJUSTE),TARJA (REAJUSTE)
0,ACETATO DE LANREOTIDA,07.718.721/0001-80,BEAUFOUR IPSEN FARMACÊUTICA LTDA,"5,39E+14","1,70E+12","7,90E+12","7,90E+12",-,SOMATULINE AUTOGEL,120 MG SOL INJ LIB PROL CT ENV AL/PLAS SER PRE...,H1C2 - HORMÔNIOS ANTICRESCIMENTO,Novo,Regulado,"2456,35","2456,35","2791,31","2959,46","2959,46","2977,39","2977,39","2995,55","2995,55","3070,44","3395,76","3858,82","4091,28","4091,28","4116,07","4116,07","4141,17","4141,17","4244,7",Não,Sim,Sim,Não,,Positiva,Não,Tarja -(*)
1,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 20,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,"57,69","57,69","65,56","69,51","69,51","69,93","69,93","70,35","70,35","72,11","79,75","90,63","96,09","96,09","96,67","96,67","97,25","97,25","99,69",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*)
2,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 30,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,"86,54","86,54","98,34","104,27","104,27","104,9","104,9","105,54","105,54","108,18","119,64","135,95","144,15","144,15","145,02","145,02","145,9","145,9","149,55",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*)
3,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 50,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,"144,23","144,23","163,9","173,77","173,77","174,82","174,82","175,89","175,89","180,29","199,39","226,58","240,23","240,23","241,68","241,68","243,16","243,16","249,24",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*)
4,ACTAEA RACEMOSA L.,78.950.011/0001-20,HERBARIUM LABORATORIO BOTANICO LTDA,"5,11E+14","1,19E+12","7,90E+12",-,-,CLIFEMIN,160 MG COM REV CT BL AL PLAS PVDC TRANS X 15,G3J - MODULADORES SELETIVOS DO RECEPTOR DE EST...,Fitoterápico,Liberado,"34,1","38,19","44,13","47,19","41,08","47,52","41,33","47,86","41,58","49,24","51,23","58,95","62,9","56,79","63,33","57,14","63,76","57,48","65,54",Não,Não,Não,Não,,Negativa,Não,Tarja -(*)


**Temos aqui as bases das últimas atualizações de medicamento no país, para inicia nossa EDA vamos verificar de quanto foi o aumento dos médicamentos.**

## Dicionário das variáveis 

**ANÁLISE RECURSAL** – Destina-se a prestar informações sobre produtos cuja análise de
preço ainda esteja em curso no âmbito da CMED, tanto em sede de pedido de reconsideração
como de recurso ao Comitê Técnico Executivo - CTE/CMED.

**APRESENTAÇÃO** – É a descrição de como o medicamento é apresentado, quanto a sua
forma farmacêutica, dosagem e quantidade.

**CAP** – O Coeficiente de Adequação de Preços é um desconto mínimo obrigatório,
atualizado anualmente, a ser aplicado sempre que forem realizadas vendas de medicamentos,
constantes do rol divulgado pela CMED e nas compras de todos os medicamentos por força de
decisão judicial, destinadas aos entes da administração pública direta e indireta da União, dos
Estados, do Distrito Federal e dos Municípios. Sobre ele, vamos falar mais à frente.

**CLASSE TERAPÊUTICA** – É a Classificação Anatômica de Produtos Farmacêuticos. Está
relacionado com a finalidade e o local de atuação do medicamento de acordo com a Associação
Europeia de Pesquisa de Mercado Farmacêutico (EphMRA).

**CNPJ** – Código de identificação da pessoa jurídica. 

**CÓDIGO GGREM** – É um campo específico de identificação de determinada
apresentação, cujo fornecimento e geração são exclusivos da CMED e são únicos para cada
apresentação.

**COMERCIALIZAÇÃO 2017** – Refere-se à informação se o produto foi ou não
comercializado pelo detentor do registro, no ano de 2017.

**CONFAZ 87** – Lista de medicamentos que ficam isentas do ICMS, cujas operações sejam
destinadas a órgãos da Administração Pública Direta e Indireta Federal, Estadual e Municipal e
do Distrito Federal e a suas fundações públicas.

**EAN 1** – O Código EAN é também conhecido como Código de Barras ou GTIN. É composto
por 13 dígitos e nele se identifica o País de origem do produto, a empresa fabricante, o produto
por ela produzido e um digito verificador.

**EAN 2** – O Código EAN é também conhecido como Código de Barras ou GTIN. É composto
por 13 dígitos e nele se identifica o País de origem do produto, a empresa fabricante, o produto
por ela produzido e um digito verificador.

**EAN 3** – O Código EAN é também conhecido como Código de Barras ou GTIN. É composto
por 13 dígitos e nele se identifica o País de origem do produto, a empresa fabricante, o produto
por ela produzido e um digito verificador.

**LABORATÓRIO** – É o nome da empresa detentora do registro sanitário podendo ser a
produtora ou importadora do medicamento.

**LISTA DE CONCESSÃO DE CRÉDITO TRIBUTÁRIO (PIS/COFINS)** – Classificação fiscal dos
medicamentos em uma das listas a seguir, conforme o regime tributário previsto na Lei nº
10.147, de 2000:

- **Lista Positiva:** medicamentos cujas substâncias ativas constam do anexo do Decreto nº 3.803, de 24 de abril de 2001, e suas atualizações, sujeitos à prescrição médica, identificados com tarja vermelha ou preta, e cujas empresas produtoras usufruem do regime especial de utilização de crédito presumido de PIS/Pasep e COFINS, classificados em três categorias, ou seja, medicamentos isentos de PIS/Pasep e COFINS:
    - I.Categoria I - Medicamentos monodroga identificados com tarja vermelha ou preta; 
    - II.Categoria II - Medicamentos em associações identificados com tarja vermelha ou preta;
    - III.Categoria III - Substâncias para medicamentos utilizados em nutrição parenteral, hemodiálise e diálise peritoneal, substitutos do plasma e       expansores plasmáticos, identificados com tarja vermelha. 
- **Lista Negativa:** medicamentos pertencentes às classificações constantes do art. 1º daLei nº 10.147, de 2000, cujas substâncias ativas não estejam relacionadas no anexo do Decreto nº 3.803, de 2001, e suas atualizações, ou seja, medicamentos incidência de 12% de PIS/Pasep e COFINS.
- **Lista Neutra:** medicamentos que não estão sujeitos ao regime tributário estabelecidona Lei nº 10.147, de 2000, ou seja, medicamentos incidência de 9,25% de PIS/Pasep e COFINS.

**PF** - PF 0%, PF 12%, PF 17%, PF 17% ALC, PF 17,5%, PF 17,5% ALC, PF 18%, PF 18% ALC, PF
20%, PF Sem Impostos, Preço Fábrica – É o teto de preço pelo qual um laboratório ou
distribuidor de medicamentos pode comercializar no mercado brasileiro um medicamento.
Desta maneira, o PF vem a ser o preço máximo permitido para vendas de medicamentos
destinadas a farmácias, drogarias, além das destinadas a entes da Administração Pública.

**PMC** - PMC 0%, PMC 12%, PMC 17%, PMC 17% ALC, PMC 17,5%, PMC 17,5% ALC, PMC 18%,
PMC 18% ALC, PMC 20%, Preço Máximo ao Consumidor ou PMC – É o preço a ser praticado pelo
comércio varejista de medicamentos, ou seja, farmácias e drogarias, tendo em vista que este
contempla tanto a margem de lucro como os impostos inerentes a esses tipos de comércio.

*A Alíquota de 0% se refere ao preço sem ICMS, que deve ser aplicado aos medicamentos isentos
de ICMS, que são aqueles relacionados nos Convênios do CONFAZ, por exemplo CONVÊNIO ICMS
162/94 que isenta medicamentos destinados ao tratamento de câncer.*

*A Alíquota de ICMS 20% se refere atualmente apenas ao Estado do RJ.*

*A Alíquota de ICMS 18% se refere aos Estados de AM, AP, BA, CE, MA, MG, PB, PE, PI, PR, RN,
RS, SE, SP, TO.*

*A Alíquota de ICMS 17,5% se refere ao Estado de RO*

*A alíquota de ICMS 17% está relacionada aos demais Estados*

*A Alíquota de ICMS 12% está relacionada com os Medicamentos Genéricos de SP e MG.*

*As Alíquotas de 17%, 17,5% e 18% que tem ao lado a sigla ALC, significa Áreas de Livre Comércio,
e dizem respeito às seguintes cidades: Manaus/Tabatinga (AM), Boa Vista/Bonfim (RR),
Macapá/Santana (AP), Guajará-Mirim (RO), Brasiléia/Epitaciolândia/ Cruzeiro do Sul (AC).*

*Além das alíquotas citadas anteriormente também é publicado o PF Sem Impostos que são os
preços sem o ICMS e sem PIS/Pasep e COFINS.*

**PRINCÍPIO ATIVO** – É uma substância química ativa, fármaco, droga ou matéria-prima
que tenha propriedades farmacológicas com finalidade medicamentosa, utilizada para
diagnóstico, alívio ou tratamento, empregada para modificar ou explorar sistemas fisiológicos
ou estados patológicos, em benefício da pessoa na qual se administra. Portanto, princípio ativo
do medicamento é o nome principal de sua composição.

**PRODUTO** – É o nome comercial dado ao medicamento, em determinada apresentação,
de um referido laboratório, no caso dos medicamentos genéricos é igual ao Princípio Ativo.

**REGISTRO** – É o número de registro de produto junto à ANVISA.

**RESTRIÇÃO HOSPITALAR** – Indica os medicamentos em embalagens hospitalares e de
uso restrito a hospitais e clínicas e, portanto, não podem ser comercializados pelo Preço Máximo
ao Consumidor não tendo seus PMC publicados.

**TARJA** – Indica qual se a venda do medicamento necessita de recita, sendo classificado em: 

- **Tarja Vermelha:** Medicamentos que só podem ser dispensados mediante apresentação de receita.
- **Tarja Preta:** Medicamentos que devem ser dispensados apenas mediante receituário acompanhado de notificação de receita, devendo ficar uma cópia no estabelecimento que realizou a dispensação.
- **Venda livre:** Medicamentos que podem ser dispensados sem a apresentação de receita.

**TIPO DE PRODUTO (STATUS DO PRODUTO)** – Tipo é a categorização do medicamento
por tipo de produto em Biológicos, Biológico Novo, Similar, Genérico, Novo, Específico,
Radiofármaco.

Status do produto indica se ele é referência.

**ICMS 0%** – Indica se o medicamento tem ou não o imposto sobre Circulação de
Mercadorias e Prestação de Serviços (ICMS), onde o imposto de cada Estado deve ser
considerado, pois ele influencia na diferença dos preços dos medicamentos de um Estado para
o outro. Por exemplo, no Estado do Paraná o ICMS é de 18% (dezoito por cento), no Estado do
Rio de Janeiro é de 20% (vinte por cento) e no Estado do Goiás é de 17% (dezessete por cento).

### Com a alta demanda de internações de pacientes da Covid-19 em UTIs (Unidade de Terapia Intensiva) de hospitais, o uso de sedativos e bloqueadores neuromusculares, necessários para intubação de pacientes da doença, estão em falta no mercado.

**Iremos iníciar nossa analise pelo:**
- Anestésico local **Lidocaína** (2%) sem vasoconstritor: esse medicamento é um anestésico utilizado no primeiro atendimento do paciente, quando ele apresenta insuficiência respiratória. É essencial e não pode ser substituído.

In [5]:
lidocaina = anterior.loc[anterior['PRODUTO'] == 'LIDOCAÍNA']
lidocaina

,SUBSTANCIA,CNPJ,LABORATORIO,CODIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTACAO,CLASSE TERAPEUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PRECO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRICAO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANALISE RECURSAL,LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS),COMERCIALIZACAO 2019,TARJA
17104,LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,40 MG/G CREM DERM CT BG AL X 30 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"51,1","51,1","58,07","61,57","61,57","61,94","61,94","62,32","62,32","63,88","70,64","80,28","85,12","85,12","85,63","85,63","86,15","86,15","88,31",Năo,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha (*)
17105,LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,62","8,62","9,8","10,39","10,39","10,45","10,45","10,51","10,51","10,78","11,92","13,55","14,36","14,36","14,45","14,45","14,53","14,53","14,9",Năo,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha
17106,LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G (SBR LAR),N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,24","8,24","9,36","9,93","9,93","9,99","9,99","10,05","10,05","10,3","11,39","12,94","13,73","13,73","13,81","13,81","13,89","13,89","14,24",Năo,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha
17108,LIDOCAÍNA,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,LIDOCAÍNA,40 MG/G CREM DERM CT BG AL X 30 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"51,13","51,13","58,1","61,6","61,6","61,98","61,98","62,35","62,35","63,91","70,68","80,32","85,16","85,16","85,68","85,68","86,2","86,2","88,35",Năo,Năo,Năo,Năo,,Positiva,Năo,Tarja Vermelha
17109,LIDOCAÍNA,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G (SBR LAR),N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,24","8,24","9,36","9,93","9,93","9,99","9,99","10,05","10,05","10,3","11,39","12,94","13,73","13,73","13,81","13,81","13,89","13,89","14,24",Năo,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha


In [6]:
lidocaina_reajuste = reajuste.loc[reajuste['PRODUTO (REAJUSTE)'] == 'LIDOCAÍNA']
lidocaina_reajuste

,SUBSTANCIA (REAJUSTE),CNPJ (REAJUSTE),LABORATORIO (REAJUSTE),CODIGO GGREM (REAJUSTE),REGISTRO (REAJUSTE),EAN 1 (REAJUSTE),EAN 2 (REAJUSTE),EAN 3 (REAJUSTE),PRODUTO (REAJUSTE),APRESENTACAO (REAJUSTE),CLASSE TERAPEUTICA (REAJUSTE),TIPO DE PRODUTO (STATUS DO PRODUTO) (REAJUSTE),REGIME DE PRECO (REAJUSTE),PF Sem Impostos (REAJUSTE),PF 0% (REAJUSTE),PF 12% (REAJUSTE),PF 17% (REAJUSTE),PF 17% ALC (REAJUSTE),"PF 17,5% (REAJUSTE)","PF 17,5% ALC (REAJUSTE)",PF 18% (REAJUSTE),PF 18% ALC (REAJUSTE),PF 20% (REAJUSTE),PMC 0% (REAJUSTE),PMC 12% (REAJUSTE),PMC 17% (REAJUSTE),PMC 17% ALC (REAJUSTE),"PMC 17,5% (REAJUSTE)","PMC 17,5% ALC (REAJUSTE)",PMC 18% (REAJUSTE),PMC 18% ALC (REAJUSTE),PMC 20% (REAJUSTE),RESTRICAO HOSPITALAR (REAJUSTE),CAP (REAJUSTE),CONFAZ 87 (REAJUSTE),ICMS 0% (REAJUSTE),ANALISE RECURSAL (REAJUSTE),LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS) (REAJUSTE),COMERCIALIZACAO 2019 (REAJUSTE),TARJA (REAJUSTE)
8251,CLORIDRATO DE LIDOCAINA,02.501.297/0001-02,PHARLAB INDÚSTRIA FARMACÊUTICA S.A.,"5,28E+14","1,41E+12","7,90E+12",-,-,LIDOCAÍNA,20 MG/G GELE TOP CT BG AL X 30 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"14,77","14,77","16,78","17,8","17,8","17,9","17,9","18,01","18,01","18,46","20,42","23,2","24,61","24,61","24,75","24,75","24,9","24,9","25,52",Não,Não,Não,Não,,Positiva,Não,Tarja -(*)
17456,LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,40 MG/G CREM DERM CT BG AL X 30 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"51,1","51,1","58,07","61,57","61,57","61,94","61,94","62,32","62,32","63,88","70,64","80,28","85,12","85,12","85,63","85,63","86,15","86,15","88,31",Não,Não,Não,Não,,Positiva,Sim,Tarja Vermelha(*)
17457,LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,62","8,62","9,8","10,39","10,39","10,45","10,45","10,51","10,51","10,78","11,92","13,55","14,36","14,36","14,45","14,45","14,53","14,53","14,9",Não,Não,Não,Não,,Positiva,Sim,Tarja Vermelha(*)
17458,LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G (SBR LAR),N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,24","8,24","9,36","9,93","9,93","9,99","9,99","10,05","10,05","10,3","11,39","12,94","13,73","13,73","13,81","13,81","13,89","13,89","14,24",Não,Não,Não,Não,,Positiva,Sim,Tarja Vermelha(*)
17460,LIDOCAÍNA,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,LIDOCAÍNA,40 MG/G CREM DERM CT BG AL X 30 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"51,13","51,13","58,1","61,6","61,6","61,98","61,98","62,35","62,35","63,91","70,68","80,32","85,16","85,16","85,68","85,68","86,2","86,2","88,35",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha
17461,LIDOCAÍNA,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G (SBR LAR),N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,24","8,24","9,36","9,93","9,93","9,99","9,99","10,05","10,05","10,3","11,39","12,94","13,73","13,73","13,81","13,81","13,89","13,89","14,24",Não,Não,Não,Não,,Positiva,Sim,Tarja Vermelha
17464,LIDOCAÍNA,02.501.297/0001-02,PHARLAB INDÚSTRIA FARMACÊUTICA S.A.,"5,28E+14","1,41E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG / G POM DERM CT BG AL X 25 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,61","8,61","9,78","10,37","10,37","10,44","10,44","10,5","10,5","10,76","11,9","13,52","14,34","14,34","14,43","14,43","14,52","14,52","14,88",Não,Não,Não,Não,,Positiva,Não,Tarja -(*)


In [7]:
merge_lidocaina = lidocaina_reajuste.merge(lidocaina, how = 'left', left_on = ['CNPJ (REAJUSTE)', 'APRESENTACAO (REAJUSTE)'], right_on = ['CNPJ', 'APRESENTACAO'])

In [8]:
merge_lidocaina

,SUBSTANCIA (REAJUSTE),CNPJ (REAJUSTE),LABORATORIO (REAJUSTE),CODIGO GGREM (REAJUSTE),REGISTRO (REAJUSTE),EAN 1 (REAJUSTE),EAN 2 (REAJUSTE),EAN 3 (REAJUSTE),PRODUTO (REAJUSTE),APRESENTACAO (REAJUSTE),CLASSE TERAPEUTICA (REAJUSTE),TIPO DE PRODUTO (STATUS DO PRODUTO) (REAJUSTE),REGIME DE PRECO (REAJUSTE),PF Sem Impostos (REAJUSTE),PF 0% (REAJUSTE),PF 12% (REAJUSTE),PF 17% (REAJUSTE),PF 17% ALC (REAJUSTE),"PF 17,5% (REAJUSTE)","PF 17,5% ALC (REAJUSTE)",PF 18% (REAJUSTE),PF 18% ALC (REAJUSTE),PF 20% (REAJUSTE),PMC 0% (REAJUSTE),PMC 12% (REAJUSTE),PMC 17% (REAJUSTE),PMC 17% ALC (REAJUSTE),"PMC 17,5% (REAJUSTE)","PMC 17,5% ALC (REAJUSTE)",PMC 18% (REAJUSTE),PMC 18% ALC (REAJUSTE),PMC 20% (REAJUSTE),RESTRICAO HOSPITALAR (REAJUSTE),CAP (REAJUSTE),CONFAZ 87 (REAJUSTE),ICMS 0% (REAJUSTE),ANALISE RECURSAL (REAJUSTE),LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS) (REAJUSTE),COMERCIALIZACAO 2019 (REAJUSTE),TARJA (REAJUSTE),SUBSTANCIA,CNPJ,LABORATORIO,CODIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTACAO,CLASSE TERAPEUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PRECO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRICAO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANALISE RECURSAL,LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS),COMERCIALIZACAO 2019,TARJA
0,CLORIDRATO DE LIDOCAINA,02.501.297/0001-02,PHARLAB INDÚSTRIA FARMACÊUTICA S.A.,"5,28E+14","1,41E+12","7,90E+12",-,-,LIDOCAÍNA,20 MG/G GELE TOP CT BG AL X 30 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"14,77","14,77","16,78","17,8","17,8","17,9","17,9","18,01","18,01","18,46","20,42","23,2","24,61","24,61","24,75","24,75","24,9","24,9","25,52",Não,Não,Não,Não,,Positiva,Não,Tarja -(*),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,40 MG/G CREM DERM CT BG AL X 30 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"51,1","51,1","58,07","61,57","61,57","61,94","61,94","62,32","62,32","63,88","70,64","80,28","85,12","85,12","85,63","85,63","86,15","86,15","88,31",Não,Não,Não,Não,,Positiva,Sim,Tarja Vermelha(*),LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,40 MG/G CREM DERM CT BG AL X 30 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"51,1","51,1","58,07","61,57","61,57","61,94","61,94","62,32","62,32","63,88","70,64","80,28","85,12","85,12","85,63","85,63","86,15","86,15","88,31",Năo,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha (*)
2,LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,62","8,62","9,8","10,39","10,39","10,45","10,45","10,51","10,51","10,78","11,92","13,55","14,36","14,36","14,45","14,45","14,53","14,53","14,9",Não,Não,Não,Não,,Positiva,Sim,Tarja Vermelha(*),LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G,N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,62","8,62","9,8","10,39","10,39","10,45","10,45","10,51","10,51","10,78","11,92","13,55","14,36","14,36","14,45","14,45","14,53","14,53","14,9",Năo,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha
3,LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G (SBR LAR),N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8,24","8,24","9,36","9,93","9,93","9,99","9,99","10,05","10,05","10,3","11,39","12,94","13,73","13,73","13,81","13,81","13,89","13,89","14,24",Não,Não,Não,Não,,Positiva,Sim,Tarja Vermelha(*),LIDOCAÍNA,57.507.378/0003-65,EMS S/A,"5,08E+14","1,02E+12","7,90E+12",-,-,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G (SBR LAR),N1B3 - ANESTÉSICOS LOCAIS TÓPICOS,Genérico,Regulado,"8

In [9]:
merge_lidocaina['VALOR_REAJUSTE'] = 'NaN'

In [10]:
merge_lidocaina['PF Sem Impostos (REAJUSTE)'] = merge_lidocaina['PF Sem Impostos (REAJUSTE)'].astype('str')
merge_lidocaina['PF Sem Impostos'] = merge_lidocaina['PF Sem Impostos'].astype('str')
merge_lidocaina['PF Sem Impostos (REAJUSTE)'] = merge_lidocaina['PF Sem Impostos'].apply(lambda x: x.replace(',','.'))
merge_lidocaina['PF Sem Impostos'] = merge_lidocaina['PF Sem Impostos'].apply(lambda x: x.replace(',','.'))
merge_lidocaina['PF Sem Impostos (REAJUSTE)'] = merge_lidocaina['PF Sem Impostos (REAJUSTE)'].astype('float')
merge_lidocaina['PF Sem Impostos'] = merge_lidocaina['PF Sem Impostos'].astype('float')

In [11]:
for index, row in merge_lidocaina.iterrows():
    if row['APRESENTACAO (REAJUSTE)'] == row['APRESENTACAO']:
        merge_lidocaina.loc[index,'VALOR_REAJUSTE'] =  100*(1 - (row['PF Sem Impostos (REAJUSTE)']/row['PF Sem Impostos']))

In [12]:
verificacao = merge_lidocaina[['LABORATORIO', 'PRODUTO', 'APRESENTACAO', 'PF Sem Impostos (REAJUSTE)', 'PF Sem Impostos', 'VALOR_REAJUSTE']]
verificacao

,LABORATORIO,PRODUTO,APRESENTACAO,PF Sem Impostos (REAJUSTE),PF Sem Impostos,VALOR_REAJUSTE
0,NaN,NaN,NaN,NaN,NaN,NaN
1,EMS S/A,LIDOCAÍNA,40 MG/G CREM DERM CT BG AL X 30 G,51.10,51.10,0.0
2,EMS S/A,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G,8.62,8.62,0.0
3,EMS S/A,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G (SBR LAR),8.24,8.24,0.0
4,GERMED FARMACEUTICA LTDA,LIDOCAÍNA,40 MG/G CREM DERM CT BG AL X 30 G,51.13,51.13,0.0
5,GERMED FARMACEUTICA LTDA,LIDOCAÍNA,50 MG/G POM DERM CT BG AL X 25 G (SBR LAR),8.24,8.24,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN


**Para o medicamento Lidocaína não teve reajuste do seu valor sem impostos**